In [35]:
import gc

import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [56]:
connection_string_path = '../../.vscode/mariadb_connection.txt'
connection_string = open(connection_string_path, 'r').read()
engine = create_engine(connection_string)

In [45]:
# from sqlalchemy import inspect
# inspector = inspect(engine)
# schemas = inspector.get_schema_names()
# for schema in schemas:
#     print("schema: %s" % schema)
#     for table_name in inspector.get_table_names(schema=schema):
#         print(table_name)
#         # for column in inspector.get_columns(table_name, schema=schema):
#         #     print("Column: %s" % column)

In [79]:
query = """
select * from chunks_data
"""
data = pd.read_sql_query(query, engine)
data.shape[0], data.kaggle_id.nunique(), data.ref_link.nunique()

In [77]:
# Check how many competitions are missing
# competitions[~competitions['ref'].isin(data['ref_link'].unique())].shape

In [2]:
pd.set_option('mode.chained_assignment', None)

In [28]:
# CHUNKS_PATH = "../../data/codeblocks_2021-04-01_concatenated_cleaned_linkedwithcompetitions.csv"
# chunks = pd.read_csv(CHUNKS_PATH)
# chunks.shape

In [78]:
MARKUP_DATA_PATH = "../../data/markup_data_2021-05-26.csv"
markup_data = pd.read_csv(MARKUP_DATA_PATH)#, nrows=25)
print('Loaded: {} chunks of {} notebooks from {} competitions'.format(markup_data.shape[0], markup_data['kaggle_id'].nunique(), markup_data['competition_id'].nunique()))

In [ ]:
NOT_YET_MARKUP_DATA_PATH = "../../data/not_yet_markup_data_2021-05-26.csv"
not_yet_markup_data = pd.read_csv(NOT_YET_MARKUP_DATA_PATH)#, nrows=25)
# not_yet_markup_data.head()
print('Loaded: {} chunks of {} notebooks from {} competitions'.format(not_yet_markup_data.shape[0], not_yet_markup_data['kaggle_id'].nunique(), not_yet_markup_data['competition_id'].nunique()))
# new_notebooks.rename({'data_sources':'ref'}, axis=1, inplace=True)
# def clean_comp(string:str) -> str:
#     string = string.strip('[').strip(']').replace("'", "")
#     return string
# new_notebooks['ref'] = new_notebooks['ref'].apply(clean_comp)
# new_notebooks['ref'] = new_notebooks['ref'].apply(lambda x: x.split(',')[0].strip(' ').strip("'"))

In [17]:
NEW_NOTEBOOKS_PATH = "../../data/codeblocks_2021-04-01_concatenated.csv"
new_notebooks = pd.read_csv(NEW_NOTEBOOKS_PATH)#, nrows=25)
new_notebooks.rename({'data_sources':'ref'}, axis=1, inplace=True)
print('Parsed: {} chunks of {} notebooks from {} competitions'.format(new_notebooks.shape[0], new_notebooks['kaggle_link'].nunique(), new_notebooks['ref'].nunique()))
def clean_comp(string:str) -> str:
    string = string.strip('[').strip(']').replace("'", "")
    return string
new_notebooks['ref'] = new_notebooks['ref'].apply(clean_comp)
new_notebooks['ref'] = new_notebooks['ref'].apply(lambda x: x.split(',')[0].strip(' ').strip("'"))

Parsed: 2587074 chunks of 101071 notebooks from 22935 competitions


In [18]:
new_notebooks.drop_duplicates(inplace=True, subset=['code_block_id', 'kaggle_id'])
print(new_notebooks.shape, new_notebooks.kaggle_link.nunique())
new_notebooks.dropna(axis=0, subset=['code_block'], inplace=True)
# print(new_notebooks.shape, new_notebooks.kaggle_link.nunique())
gc.collect()
print('Parsed and cleaned: {} chunks of {} notebooks from {} competitions'.format(new_notebooks.shape[0], new_notebooks['kaggle_link'].nunique(), new_notebooks['ref'].nunique()))

(2441865, 9) 101065
Parsed and cleaned: 2441864 chunks of 101065 notebooks from 12984 competitions


In [19]:
new_notebooks.dropna(axis=0, subset=['kaggle_score'], inplace=True) 
print('With kaggle_score: {} chunks of {} notebooks from {} competitions'.format(new_notebooks.shape[0], new_notebooks['kaggle_link'].nunique(), new_notebooks['ref'].nunique()))

With kaggle_score: 617441 chunks of 23425 notebooks from 553 competitions


In [20]:
new_notebooks['code_block'] = new_notebooks['code_block'].str.replace('`', '')

In [21]:
# new_notebooks.to_csv("../../data/codeblocks_2021-04-01_concatenated_cleaned.csv", index=False)

In [22]:
COMPETITIONS_PATH = "../../data/competitions_info.csv"
competitions = pd.read_csv(COMPETITIONS_PATH)
print(competitions.shape)
competitions.drop_duplicates(inplace=True)
competitions['ref'] = competitions['ref'].apply(clean_comp)
print(competitions.shape)

(5060, 8)
(4352, 8)


In [23]:
new_notebooks = new_notebooks.merge(competitions, on=['ref'], how='inner')
# new_notebooks.merge(competitions, on=['ref'], how='right').shape
print('# Chunks: {}\n# Notebooks: {}\n# Competitions: {}'.format(new_notebooks.shape[0], new_notebooks.kaggle_link.nunique(), new_notebooks.ref.nunique()))

# Chunks: 77649
# Notebooks: 2827
# Competitions: 266


In [24]:
# Check number of "non competition" refs
new_notebooks[~new_notebooks['ref'].str.contains('c/')].shape[0]

0

In [ ]:
# new_notebooks.drop('Unnamed: 0', axis=1, inplace=True)
# new_notebooks.to_csv('../../data/codeblocks_2021-04-01_concatenated_cleaned.csv', index=False)

In [10]:
# new_notebooks['exists_in_competitions'] = new_notebooks.apply(lambda x: x['ref'] in competitions['ref'].unique(), axis=1)
# print('# Chunks: {}\n# Notebooks: {}\n# Competitions: {}'.format(new_notebooks[new_notebooks['exists_in_competitions'] == True].shape[0], new_notebooks[new_notebooks['exists_in_competitions'] == True]['kaggle_link'].nunique(), new_notebooks[new_notebooks['exists_in_competitions'] == True]['ref'].nunique()))

# Chunks: 77649
# Notebooks: 2827
# Competitions: 266


In [ ]:
competitions['has_notebooks'] = competitions.apply(lambda x: x['ref'] in new_notebooks['ref'].tolist(), axis=1)
competitions_cleaned = competitions[competitions['has_notebooks']]
competitions_cleaned.shape
# competitions_cleaned.to_csv('../data/competitions_info_cleaned.csv', index=False)

In [ ]:
COMPETITIONS_FILLED_PATH = "../../data/competitions_info_cleaned.csv"
competitions_filled = pd.read_csv(COMPETITIONS_FILLED_PATH)
competitions_filled.shape

In [ ]:
competitions_cleaned_with_filled = competitions_cleaned[['ref', 'comp_name', 'comp_type']].merge(competitions_filled[['ref', 'Description',
                                        'Metric', 'DataType', 'Subject', 'ProblemType']]
                                    , on=['ref'], how='left')
competitions_cleaned_with_filled.isna().sum()

In [ ]:
# competitions_filled.to_csv('../../data/competitions_info_filled.csv', index=False)
# competitions_filled.to_csv('../../data_mini/competitions_info_filled.csv', index=False)

In [ ]:
# competitions_cleaned.to_csv('../../data/competitions_info_cleaned.csv', index=False)
# competitions_cleaned.to_csv('../../data_mini/competitions_info_cleaned.csv', index=False)

In [ ]:
# competitions_cleaned_with_filled.to_csv('../../data/competitions_info_cleaned_filled.csv', index=False)
# competitions_cleaned_with_filled.to_csv('../../data_mini/competitions_info_cleaned_filled.csv', index=False)